In [1]:
from ltr.client import OpenSearchClient
client = OpenSearchClient()

In [2]:
from ltr import download
corpus='http://es-learn-to-rank.labs.o19s.com/tmdb.json'
judgments='http://es-learn-to-rank.labs.o19s.com/title_judgments.txt'

download([corpus, judgments], dest='data/');

data/tmdb.json already exists
data/title_judgments.txt already exists


In [8]:
from ltr.index import rebuild
from ltr.helpers.movies import indexable_movies

movies=indexable_movies(movies='data/tmdb.json')
force=False
rebuild(client, index='tmdb', doc_src=movies, force)

Deleted index tmdb [Status: 200]
Created index tmdb [Status: 200]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 27846/27846 [00:43<00:00, 637.22it/s]


Streaming Bulk index DONE tmdb [Status: 201]


In [9]:
client.reset_ltr(index='tmdb')

config = {"validation": {
              "index": "tmdb",
              "params": {
                  "keywords": "rambo"
              }
    
           },
           "featureset": {
            "features": [
                { #1
                    "name": "title_bm25",
                    "params": ["keywords"],
                    "template": {
                        "match": {"title": "{{keywords}}"}
                    }
                },
                { #2
                    "name": "overview_bm25",
                    "params": ["keywords"],
                    "template": {
                        "match": {"overview": "{{keywords}}"}
                    }
                }
            ]
    }}


client.create_featureset(index='tmdb', name='movies', ftr_config=config)

Removed Default LTR feature store [Status: 200]
Initialize Default LTR feature store [Status: 200]
Create movies feature set [Status: 201]


In [10]:
from ltr.log import FeatureLogger
from ltr.judgments import judgments_open
from itertools import groupby

ftr_logger=FeatureLogger(client, index='tmdb', feature_set='movies')
with judgments_open('data/title_judgments.txt') as judgment_list:
    for qid, query_judgments in groupby(judgment_list, key=lambda j: j.qid):
        ftr_logger.log_for_qid(judgments=query_judgments, 
                               qid=qid,
                               keywords=judgment_list.keywords(qid))

Recognizing 40 queries


In [11]:
from ltr.ranklib import train
trainLog  = train(client,
                  training_set=ftr_logger.logged,
                  index='tmdb',
                  featureSet='movies',
                  modelName='title')

/var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/RankyMcRankFace.jar already exists
Running java -jar /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/RankyMcRankFace.jar -ranker 6 -shrinkage 0.1 -metric2t DCG@10 -tree 50 -bag 1 -leaf 10 -frate 1.0 -srate 1.0 -train /var/folders/33/jx0mw87156q2hmtrr_r82s7r0000gs/T/training.txt -save data/title_model.txt 
Delete model title: 404
Created Model title [Status: 201]
Model saved


In [12]:
from ltr import search
search(client, "cartoon with basketball aliens", modelName='title')

{'fl': '*,score', 'rows': 5, 'q': '{!ltr reRankDocs=30000 model=title efi.keywords="cartoon with basketball aliens" efi.fuzzy_keywords="cartoon~ with~ basketball~ aliens~"}'}


RequestError: RequestError(400, 'parsing_exception', 'Unknown key for a VALUE_STRING in [fl].')